In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import matplotlib
import base

myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [ ]:
def initArimaData(id):
    open_time = shop_open_dates.loc[id]['date'];
    if open_time < pd.to_datetime('2015-7-1'):
        open_time = pd.to_datetime('2015-7-1')
    df = base.countShopPayTimePeriods(user_pay_counts, id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    
    return df

In [ ]:
arima_df = initArimaData(id)
arima_df.to_csv("./data/dataset/dataset/arima/" + str(id), index=False, encoding='UTF-8')

In [ ]:
arima_df = pd.read_csv("./data/dataset/dataset/arima/" + str(id) + "_result", encoding='UTF-8', index_col=0)
arima_df['x'] = arima_df['x'].astype(np.int32)

In [ ]:
y_arima = arima_df['x'].values
e = base.evaluation(y_Test, y_arima)
print("id=%d evaluation=%f"%(id, e))
print(y_Test)
print(y_arima)
PlotPredAndTestPayCount(y_Test, y_arima, train_mean)

In [ ]:
for id in range(1, 2001):
    arima_df = initArimaData(id)
    arima_df.to_csv("./data/dataset/dataset/arima/" + str(id), index=False, encoding='UTF-8')

In [ ]:
#ARIMA 时间序列预测
evaluations = {}
arima_df = base.ReadArimaPayCountCSV()

for id in range(1,2001):
    y_arima = arima_df.ix[id].values
    y_Test = base.getTestCount(user_pay_counts, id, pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')).values
    e = base.evaluation_tm(y_Test, y_arima)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e
    
arima_evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
arima_evaluation_df.columns = ['evaluation']
arima_evaluation_df.index.name = 'id'
arima_evaluation_df.sort_values('evaluation')
arima_evaluation_df.to_csv("./data/dataset/dataset/arima_evaluation.txt", sep='\t', index=True, encoding='UTF-8')